In [1]:
import GWFish.modules as gw

from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import corner
import numpy as np
import pandas as pd
import json
import os
from astropy.cosmology import Planck18

GWFish_path = '/home/anik/anaconda3/envs/tgr/lib/python3.9/site-packages/GWFish/'

### When in Colab use instead:
#GWFish_path = '/usr/local/lib/python3.10/dist-packages/GWFish/'

In [2]:
population = 'BBH' # or 'BBH'
detectors = ['LHO']
networks = '[[0]]'

detectors_ids = detectors
networks_ids = json.loads(networks)
ConfigDet = os.path.join(GWFish_path,'detectors.yaml')

calculate_errors = True
duty_cycle = False

waveform_model = 'TaylorF2'
waveform_class = gw.waveforms.LALFD_Waveform

In [3]:
z = np.array([0])

parameters = {
    'mass_1': np.array([31.4957673]), 
    'mass_2': np.array([21.24276395]), 
    'redshift': z,
    'luminosity_distance': np.array([440]),
    'theta_jn': np.array([2.545065595974997]),
    'ra': np.array([3.4461599999999994]), # phi in gwfast
    'dec': np.array([-0.4080839999999999]), # pi/2 - theta
    'psi': np.array([0.]),
    'phase': np.array([0.]),
    'geocent_time': np.array([1187008882.4]),
    'a_1':np.array([0.005136138323169717]), 
    'a_2':np.array([0.003235146993487445]), 
    'lambda_1':np.array([368.17802383555687]), 
    'lambda_2':np.array([586.5487031450857]),
    'eccentricity': np.array([0.1]),
    'min_frequency_cutoff':10}
parameters = pd.DataFrame(parameters)
parameters

,mass_1,mass_2,redshift,luminosity_distance,theta_jn,ra,dec,psi,phase,geocent_time,a_1,a_2,lambda_1,lambda_2,eccentricity,min_frequency_cutoff
0,31.495767,21.242764,0,440,2.545066,3.44616,-0.408084,0.0,0.0,1.187009e+09,0.005136,0.003235,368.178024,586.548703,0.1,10


In [4]:
threshold_SNR = np.array([0., 8.])
fisher_parameters = ['mass_1']
network = gw.detection.Network(detectors_ids, detection_SNR=threshold_SNR, parameters=parameters,
                               fisher_parameters=fisher_parameters, config=ConfigDet)


### Calculate SNR

In [5]:
for k in tqdm(np.arange(len(parameters))):
    parameter_values = parameters.iloc[k]

    networkSNR_sq = 0
    for d in np.arange(len(network.detectors)):
        data_params = {
            'frequencyvector': network.detectors[d].frequencyvector,
            'f_ref': 20.
        }
        waveform_obj = waveform_class(waveform_model, parameter_values, data_params)
        wave = waveform_obj()
        t_of_f = waveform_obj.t_of_f

        signal = gw.detection.projection(parameter_values, network.detectors[d], wave, t_of_f)

        SNRs = gw.detection.SNR(network.detectors[d], signal, duty_cycle=duty_cycle)
        networkSNR_sq += np.sum(SNRs ** 2)
        network.detectors[d].SNR[k] = np.sqrt(np.sum(SNRs ** 2))

        if calculate_errors:
            network.detectors[d].fisher_matrix[k, :, :] = \
                gw.fishermatrix.FisherMatrix(waveform_model, parameter_values, fisher_parameters, network.detectors[d], waveform_class=waveform_class).fm

    network.SNR[k] = np.sqrt(networkSNR_sq)

gw.detection.analyzeDetections(network, parameters, population, networks_ids)

if calculate_errors:
    gw.fishermatrix.analyzeFisherErrors(network, parameters, fisher_parameters, population, networks_ids)


print('The network SNR of the event is ', network.SNR[0])
errors = pd.read_csv('outdir/Errors_LHO_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
print(errors.iloc[0])

100%|██████████| 1/1 [00:00<00:00, 16.42it/s]

Network: LHO
Detected signals with SNR>8.000: 1.000 (1 out of 1); z<0.000
SNR: 118.623 (min) , 118.623 (max) 
The network SNR of the event is  118.6232527706405
network_SNR             1.186233e+02
mass_1                  3.150000e+01
mass_2                  2.124000e+01
redshift                0.000000e+00
luminosity_distance     4.400000e+02
theta_jn                2.545000e+00
ra                      3.446000e+00
dec                    -4.081000e-01
psi                     0.000000e+00
phase                   0.000000e+00
geocent_time            1.187000e+09
a_1                     5.136000e-03
a_2                     3.235000e-03
lambda_1                3.682000e+02
lambda_2                5.865000e+02
eccentricity            1.000000e-01
min_frequency_cutoff    1.000000e+01
err_mass_1              4.229000e-04
Name: 0, dtype: float64


In [6]:
z = np.array([0])

parameters = {
    'mass_1': np.array([31.4957673]), 
    'mass_2': np.array([21.24276395]), 
    'redshift': z,
    'distance': np.array([440]),
    'inclination': np.array([2.545065595974997]),
    'ra': np.array([3.4461599999999994]), # phi in gwfast
    'dec': np.array([-0.4080839999999999]), # pi/2 - theta
    'psi': np.array([0.]),
    'phase': np.array([0.]),
    'geocent_time': np.array([1187008882.4]),
    'spin1z':np.array([0.005136138323169717]), 
    'spin2z':np.array([0.003235146993487445]), 
    'min_frequency_cutoff': np.array([20]),
    'max_frequency_cutoff':np.array([1024])
}
parameters = pd.DataFrame(parameters)
parameters

,mass_1,mass_2,redshift,distance,inclination,ra,dec,psi,phase,geocent_time,spin1z,spin2z,min_frequency_cutoff,max_frequency_cutoff
0,31.495767,21.242764,0,440,2.545066,3.44616,-0.408084,0.0,0.0,1.187009e+09,0.005136,0.003235,20,1024


In [7]:
waveform_model = 'TaylorF2'
waveform_class = gw.waveforms.PyCBC_Waveform

threshold_SNR = np.array([0., 8.])
fisher_parameters = ['mass_1']
network = gw.detection.Network(detectors_ids, detection_SNR=threshold_SNR, parameters=parameters,
                               fisher_parameters=fisher_parameters, config=ConfigDet)


In [8]:
for k in tqdm(np.arange(len(parameters))):
    parameter_values = parameters.iloc[k]

    networkSNR_sq = 0
    for d in np.arange(len(network.detectors)):
        data_params = {
            'frequencyvector': network.detectors[d].frequencyvector,
            'f_ref': 20.
        }
        waveform_obj = waveform_class(waveform_model, parameter_values, data_params)
        wave = waveform_obj()
        t_of_f = waveform_obj.t_of_f

        signal = gw.detection.projection(parameter_values, network.detectors[d], wave, t_of_f)

        SNRs = gw.detection.SNR(network.detectors[d], signal, duty_cycle=duty_cycle)
        networkSNR_sq += np.sum(SNRs ** 2)
        network.detectors[d].SNR[k] = np.sqrt(np.sum(SNRs ** 2))

        if calculate_errors:
            network.detectors[d].fisher_matrix[k, :, :] = \
                gw.fishermatrix.FisherMatrix(waveform_model, parameter_values, fisher_parameters, network.detectors[d], waveform_class=waveform_class).fm

    network.SNR[k] = np.sqrt(networkSNR_sq)

gw.detection.analyzeDetections(network, parameters, population, networks_ids)

if calculate_errors:
    gw.fishermatrix.analyzeFisherErrors(network, parameters, fisher_parameters, population, networks_ids)


print('The network SNR of the event is ', network.SNR[0])
errors = pd.read_csv('outdir/Errors_LHO_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
print(errors.iloc[0])

100%|██████████| 1/1 [00:00<00:00,  2.89it/s]

Network: LHO
Detected signals with SNR>8.000: 1.000 (1 out of 1); z<0.000
SNR: 116.978 (min) , 116.978 (max) 
The network SNR of the event is  116.97795556109955
network_SNR             1.169780e+02
mass_1                  3.150000e+01
mass_2                  2.124000e+01
redshift                0.000000e+00
distance                4.400000e+02
inclination             2.545000e+00
ra                      3.446000e+00
dec                    -4.081000e-01
psi                     0.000000e+00
phase                   0.000000e+00
geocent_time            1.187000e+09
spin1z                  5.136000e-03
spin2z                  3.235000e-03
min_frequency_cutoff    2.000000e+01
max_frequency_cutoff    1.024000e+03
err_mass_1              1.320000e-02
Name: 0, dtype: float64


## fisher ecc

In [9]:
z = np.array([0])

parameters = {
    'mass_1': np.array([31.4957673]), 
    'mass_2': np.array([21.24276395]), 
    'eccentricity': np.array([0.1]),
    'redshift': z,
    'distance': np.array([440]),
    'inclination': np.array([2.545065595974997]),
    'ra': np.array([3.4461599999999994]), # phi in gwfast
    'dec': np.array([-0.4080839999999999]), # pi/2 - theta
    'psi': np.array([0.]),
    'phase': np.array([0.]),
    'geocent_time': np.array([1187008882.4]),
    'spin1z':np.array([0.005136138323169717]), 
    'spin2z':np.array([0.003235146993487445]), 
    'min_frequency_cutoff': np.array([20]),
    'max_frequency_cutoff':np.array([1024])
}
parameters = pd.DataFrame(parameters)
parameters

,mass_1,mass_2,eccentricity,redshift,distance,inclination,ra,dec,psi,phase,geocent_time,spin1z,spin2z,min_frequency_cutoff,max_frequency_cutoff
0,31.495767,21.242764,0.1,0,440,2.545066,3.44616,-0.408084,0.0,0.0,1.187009e+09,0.005136,0.003235,20,1024


In [10]:
waveform_model = 'TaylorF2Ecc'
waveform_class = gw.waveforms.PyCBC_Waveform

threshold_SNR = np.array([0., 8.])
fisher_parameters = ['eccentricity']
network = gw.detection.Network(detectors_ids, detection_SNR=threshold_SNR, parameters=parameters,
                               fisher_parameters=fisher_parameters, config=ConfigDet)


In [11]:
for k in tqdm(np.arange(len(parameters))):
    parameter_values = parameters.iloc[k]

    networkSNR_sq = 0
    for d in np.arange(len(network.detectors)):
        data_params = {
            'frequencyvector': network.detectors[d].frequencyvector,
            'f_ref': 20.
        }
        waveform_obj = waveform_class(waveform_model, parameter_values, data_params)
        wave = waveform_obj()
        t_of_f = waveform_obj.t_of_f

        signal = gw.detection.projection(parameter_values, network.detectors[d], wave, t_of_f)

        SNRs = gw.detection.SNR(network.detectors[d], signal, duty_cycle=duty_cycle)
        networkSNR_sq += np.sum(SNRs ** 2)
        network.detectors[d].SNR[k] = np.sqrt(np.sum(SNRs ** 2))

        if calculate_errors:
            network.detectors[d].fisher_matrix[k, :, :] = \
                gw.fishermatrix.FisherMatrix(waveform_model, parameter_values, fisher_parameters, network.detectors[d], waveform_class=waveform_class).fm

    network.SNR[k] = np.sqrt(networkSNR_sq)

gw.detection.analyzeDetections(network, parameters, population, networks_ids)

if calculate_errors:
    gw.fishermatrix.analyzeFisherErrors(network, parameters, fisher_parameters, population, networks_ids)


print('The network SNR of the event is ', network.SNR[0])
errors = pd.read_csv('outdir/Errors_LHO_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
print(errors.iloc[0])

100%|██████████| 1/1 [00:00<00:00,  2.19it/s]

Network: LHO
Detected signals with SNR>8.000: 1.000 (1 out of 1); z<0.000
SNR: 116.978 (min) , 116.978 (max) 
The network SNR of the event is  116.97795556109956
network_SNR             1.169780e+02
mass_1                  3.150000e+01
mass_2                  2.124000e+01
eccentricity            1.000000e-01
redshift                0.000000e+00
distance                4.400000e+02
inclination             2.545000e+00
ra                      3.446000e+00
dec                    -4.081000e-01
psi                     0.000000e+00
phase                   0.000000e+00
geocent_time            1.187000e+09
spin1z                  5.136000e-03
spin2z                  3.235000e-03
min_frequency_cutoff    2.000000e+01
max_frequency_cutoff    1.024000e+03
err_eccentricity        3.540000e-07
Name: 0, dtype: float64
